# E-Commerce Customer Segmentation Dashboard
**Author:** Vrajkumar Patel  **Date:** 2025-11-23

This portfolio-ready notebook performs RFM analysis and K-Means clustering on e-commerce transactions to identify high-value, loyal, and at-risk customers. It includes data cleaning, feature engineering, EDA, clustering, and actionable insights with clear explanations.

# Customer Segmentation for E-Commerce

End-to-end, portfolio-ready analysis that segments customers by purchasing behavior using RFM and K-Means. The notebook automates data loading, cleaning, feature engineering, EDA, clustering, and actionable insights.

## Step 0: Setup
Imports and configuration for reproducibility and visualization.

In [ ]:
# Setup: imports and environment configuration
import os
import io
import sys
import math
import datetime as dt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

try:
    import plotly.express as px
except Exception:
    px = None

np.random.seed(42)
pd.set_option('display.float_format', lambda x: f'{x:,.2f}')
sns.set(style='whitegrid', context='notebook')

print('Environment ready:', {'python': sys.version.split()[0], 'pandas': pd.__version__})

## Step 1: Data Import
Load a publicly available e-commerce dataset. This notebook uses the UCI "Online Retail" dataset (Excel) which includes: `InvoiceNo`, `CustomerID`, `Description` (Product), `Quantity`, `UnitPrice` (Price), and `InvoiceDate` (Date). If remote loading fails, synthetic data is generated to keep the notebook reproducible.

In [ ]:
# Step 1: Data import from UCI or synthetic fallback
DATA_URL = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00352/Online%20Retail.xlsx'

def load_ecommerce_data():
    df = None
    try:
        df = pd.read_excel(DATA_URL, dtype={'InvoiceNo': str, 'StockCode': str})
        print('Loaded dataset from UCI:', DATA_URL)
    except Exception as e:
        print('Remote load failed, generating synthetic dataset')
        n_customers = 500
        n_invoices = 3000
        start_date = dt.datetime(2010, 12, 1)
        end_date = dt.datetime(2011, 12, 9)
        date_range = pd.date_range(start_date, end_date, freq='H')
        cust_ids = np.random.choice(range(10000, 10000 + n_customers), n_invoices)
        invoice_ids = [str(500000 + i) for i in range(n_invoices)]
        quantities = np.random.randint(1, 10, size=n_invoices)
        prices = np.round(np.random.uniform(1.0, 50.0, size=n_invoices), 2)
        products = np.random.choice(['Widget A', 'Widget B', 'Gadget C', 'Accessory D'], size=n_invoices)
        dates = np.random.choice(date_range, size=n_invoices)
        df = pd.DataFrame({
            'InvoiceNo': invoice_ids,
            'CustomerID': cust_ids,
            'Description': products,
            'Quantity': quantities,
            'UnitPrice': prices,
            'InvoiceDate': dates
        })
    df['Invoice ID'] = df['InvoiceNo']
    df['Customer ID'] = df['CustomerID']
    df['Product'] = df['Description']
    df['Price'] = df['UnitPrice']
    df['Date'] = pd.to_datetime(df['InvoiceDate'])
    return df

raw_df = load_ecommerce_data()
display(raw_df.head())
print('Shape:', raw_df.shape)
buffer = io.StringIO()
raw_df.info(buf=buffer)
print(buffer.getvalue())
display(raw_df.describe(include='all').transpose())

## Step 2: Data Cleaning
- Handle missing values.
- Remove duplicates.
- Correct inconsistent data (negative or zero quantities/prices).
- Convert date columns to datetime.
- Aggregate fields needed for downstream analysis.

In [ ]:
# Step 2: Clean transactions, compute totals, standardize columns
def clean_data(df):
    before_rows = len(df)
    df = df.copy()
    df = df.dropna(subset=['CustomerID'])
    df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'], errors='coerce')
    df = df.dropna(subset=['InvoiceDate'])
    df = df[(df['Quantity'] > 0) & (df['UnitPrice'] > 0)]
    df = df.drop_duplicates()
    df['TotalPrice'] = df['Quantity'] * df['UnitPrice']
    df['Invoice ID'] = df['InvoiceNo']
    df['Customer ID'] = df['CustomerID'].astype(int)
    df['Product'] = df['Description']
    df['Price'] = df['UnitPrice']
    df['Date'] = df['InvoiceDate']
    after_rows = len(df)
    print('Rows before:', before_rows, 'after:', after_rows)
    return df

clean_df = clean_data(raw_df)
display(clean_df.head())
print('Unique customers:', clean_df['Customer ID'].nunique())
print('Distinct invoices:', clean_df['Invoice ID'].nunique())

## Step 3: Feature Engineering (RFM)
Compute Recency (days since last purchase), Frequency (number of invoices), Monetary (total spend), optional features such as average order value and average days between purchases, plus product preferences and seasonality signals.

In [ ]:
# Step 3: Compute customer-level RFM and seasonality features
def compute_rfm(df):
    snapshot_date = df['InvoiceDate'].max() + pd.Timedelta(days=1)
    recency = df.groupby('Customer ID')['InvoiceDate'].max().apply(lambda x: (snapshot_date - x).days)
    frequency = df.groupby('Customer ID')['Invoice ID'].nunique()
    monetary = df.groupby('Customer ID')['TotalPrice'].sum()
    aov = monetary / frequency
    def avg_days_between(group):
        dates = group['InvoiceDate'].drop_duplicates().sort_values()
        if len(dates) < 2:
            return np.nan
        deltas = dates.diff().dropna().dt.days
        return deltas.mean()
    days_between = df.groupby('Customer ID').apply(avg_days_between)
    month_series = df['InvoiceDate'].dt.month
    quarter_series = df['InvoiceDate'].dt.quarter
    weekend_series = df['InvoiceDate'].dt.weekday >= 5
    top_product = df.groupby('Customer ID')['Description'].agg(lambda s: s.value_counts().idxmax())
    peak_month = df.assign(Month=month_series).groupby('Customer ID')['Month'].agg(lambda s: s.value_counts().idxmax())
    q4_share = df.assign(Q=quarter_series).groupby('Customer ID')['Q'].apply(lambda s: (s == 4).mean())
    weekend_ratio = df.assign(Weekend=weekend_series).groupby('Customer ID')['Weekend'].mean()
    rfm = pd.DataFrame({
        'Recency': recency,
        'Frequency': frequency,
        'Monetary': monetary,
        'AvgOrderValue': aov,
        'DaysBetweenPurchases': days_between,
        'TopProduct': top_product,
        'PeakMonth': peak_month,
        'Q4Share': q4_share,
        'WeekendRatio': weekend_ratio
    })
    rfm = rfm.fillna({'DaysBetweenPurchases': rfm['DaysBetweenPurchases'].median()})
    rfm = rfm.sort_values(by=['Monetary'], ascending=False)
    print('RFM computed for customers:', len(rfm))
    return rfm

rfm = compute_rfm(clean_df)
display(rfm.head())
display(rfm.describe().transpose())

## Step 4: Exploratory Data Analysis (EDA)
Visualize distributions, relationships, and correlations among RFM features to identify patterns and customer groups.

In [ ]:
# Step 4: EDA visuals for distributions, relationships, correlations
fig, axes = plt.subplots(2, 3, figsize=(16, 9))
sns.histplot(rfm['Recency'], kde=True, ax=axes[0,0], color='steelblue')
axes[0,0].set_title('Recency (days)')
sns.histplot(rfm['Frequency'], kde=True, ax=axes[0,1], color='seagreen')
axes[0,1].set_title('Frequency (invoices)')
sns.histplot(rfm['Monetary'], kde=True, ax=axes[0,2], color='salmon')
axes[0,2].set_title('Monetary (total spend)')
sns.boxplot(y=rfm['Recency'], ax=axes[1,0], color='steelblue')
axes[1,0].set_title('Recency Boxplot')
sns.boxplot(y=rfm['Frequency'], ax=axes[1,1], color='seagreen')
axes[1,1].set_title('Frequency Boxplot')
sns.boxplot(y=rfm['Monetary'], ax=axes[1,2], color='salmon')
axes[1,2].set_title('Monetary Boxplot')
plt.tight_layout()
plt.show()

plt.figure(figsize=(6,5))
sns.scatterplot(x='Recency', y='Frequency', data=rfm, alpha=0.6)
plt.title('Recency vs Frequency')
plt.show()

plt.figure(figsize=(6,5))
sns.scatterplot(x='Frequency', y='Monetary', data=rfm, alpha=0.6)
plt.title('Frequency vs Monetary')
plt.show()

corr = rfm[['Recency','Frequency','Monetary','AvgOrderValue','DaysBetweenPurchases','Q4Share','WeekendRatio']].corr()
plt.figure(figsize=(7,6))
sns.heatmap(corr, annot=True, cmap='vlag', fmt='.2f')
plt.title('Correlation Heatmap')
plt.show()

display(corr)

## Step 5: Customer Segmentation (Clustering)
Use K-Means on standardized RFM features. Determine optimal clusters via elbow (inertia) and silhouette scores, then analyze cluster characteristics.

In [ ]:
# Step 5: K-Means clustering on standardized RFM with model selection
features = rfm[['Recency','Frequency','Monetary']].copy()
scaler = StandardScaler()
X = scaler.fit_transform(features)
ks = list(range(2, 9))
inertias = []
silhouettes = []
for k in ks:
    km = KMeans(n_clusters=k, random_state=42, n_init=10)
    km.fit(X)
    inertias.append(km.inertia_)
    labels = km.labels_
    score = silhouette_score(X, labels)
    silhouettes.append(score)

plt.figure(figsize=(12,4))
plt.subplot(1,2,1)
plt.plot(ks, inertias, '-o')
plt.title('Elbow: Inertia vs k')
plt.xlabel('k')
plt.ylabel('Inertia')
plt.subplot(1,2,2)
plt.plot(ks, silhouettes, '-o', color='orange')
plt.title('Silhouette vs k')
plt.xlabel('k')
plt.ylabel('Silhouette')
plt.tight_layout()
plt.show()

best_k = ks[int(np.argmax(silhouettes))]
print('Chosen k by silhouette:', best_k)
km_final = KMeans(n_clusters=best_k, random_state=42, n_init=10)
rfm['Cluster'] = km_final.fit_predict(X)
cluster_stats = rfm.groupby('Cluster').agg({
    'Recency':'mean',
    'Frequency':'mean',
    'Monetary':'mean',
    'AvgOrderValue':'mean',
    'DaysBetweenPurchases':'mean'
}).round(2)
cluster_stats['Size'] = rfm.groupby('Cluster').size()
display(cluster_stats)
rfm_clusters = rfm.copy()
rfm_clusters.reset_index(inplace=True)
display(rfm_clusters.head())

## Step 6: Visualization & Dashboard
Visualize clusters in 2D and summarize cluster characteristics in a table.

In [ ]:
# Step 6: 2D cluster visualization and summary table
if px is not None:
    fig = px.scatter(rfm, x='Frequency', y='Monetary', color='Cluster', title='Clusters: Frequency vs Monetary',
                     labels={'Frequency':'Frequency','Monetary':'Monetary'})
    fig.show()
    fig2 = px.scatter(rfm, x='Recency', y='Frequency', color='Cluster', title='Clusters: Recency vs Frequency',
                      labels={'Recency':'Recency','Frequency':'Frequency'})
    fig2.show()
else:
    plt.figure(figsize=(7,5))
    sns.scatterplot(data=rfm, x='Frequency', y='Monetary', hue='Cluster', palette='tab10', alpha=0.7)
    plt.title('Clusters: Frequency vs Monetary')
    plt.show()
    plt.figure(figsize=(7,5))
    sns.scatterplot(data=rfm, x='Recency', y='Frequency', hue='Cluster', palette='tab10', alpha=0.7)
    plt.title('Clusters: Recency vs Frequency')
    plt.show()

display(cluster_stats)

## Step 7: Insights & Recommendations
Interpret clusters and propose actions such as loyalty programs, re-engagement campaigns, and targeted marketing.

### Business Interpretation
- High-value: Low Recency, High Frequency, High Monetary. Recommend exclusive loyalty perks, early access, bundled offers.
- Loyal: Moderate Recency, High Frequency, Moderate Monetary. Recommend points-based rewards, referral incentives, personalized recommendations.
- At-risk: High Recency, Low Frequency, Moderate/Low Monetary. Recommend win-back emails, time-limited discounts, reactivation nudges.
- Low-value: Variable Recency, Low Frequency, Low Monetary. Recommend awareness campaigns, cross-sell essentials, optimize onboarding.

Cluster-specific actions should align with product preferences observed from `Description` and seasonal patterns in `InvoiceDate`.

## Step 8a: Persist Artifacts
Save clustered customers and cluster stats as CSV for downstream workflows.

In [ ]:
# Step 8a: Persist artifacts for dashboard and downstream workflows
out_dir = 'data'
os.makedirs(out_dir, exist_ok=True)
rfm_clusters_df.to_csv(os.path.join(out_dir, 'rfm_clusters.csv'), index=False)
rfm_cluster_stats.to_csv(os.path.join(out_dir, 'rfm_cluster_stats.csv'))
print('Saved:', os.path.join(out_dir, 'rfm_clusters.csv'))
print('Saved:', os.path.join(out_dir, 'rfm_cluster_stats.csv'))

## Step 8: End-to-End Automation
Pipeline function to run the full workflow and return clustered RFM data with stats for reuse and deployment.

In [ ]:
# Step 8: End-to-end pipeline to compute clusters and stats
def run_pipeline():
    d0 = load_ecommerce_data()
    d1 = clean_data(d0)
    r = compute_rfm(d1)
    feats = r[['Recency','Frequency','Monetary']]
    sc = StandardScaler()
    X_ = sc.fit_transform(feats)
    ks_ = list(range(2, 9))
    sils_ = []
    for k in ks_:
        km_ = KMeans(n_clusters=k, random_state=42, n_init=10).fit(X_)
        sils_.append(silhouette_score(X_, km_.labels_))
    kstar = ks_[int(np.argmax(sils_))]
    km_ = KMeans(n_clusters=kstar, random_state=42, n_init=10).fit(X_)
    r['Cluster'] = km_.labels_
    stats = r.groupby('Cluster').agg({
        'Recency':'mean','Frequency':'mean','Monetary':'mean','AvgOrderValue':'mean','DaysBetweenPurchases':'mean'
    }).round(2)
    stats['Size'] = r.groupby('Cluster').size()
    return r.reset_index(), stats, kstar

rfm_clusters_df, rfm_cluster_stats, chosen_k = run_pipeline()
print('Pipeline complete. Chosen k:', chosen_k)
display(rfm_cluster_stats)
display(rfm_clusters_df.head())

## Conclusion & Recommendations
- High-value segments (low R, high F/M) benefit from loyalty rewards and premium offers.
- At-risk segments (high R) respond to win-back campaigns and time-limited discounts.
- Product preferences (TopProduct) and seasonal signals (PeakMonth) enable tailored promotions.
- This analysis supports targeted marketing, retention strategies, and revenue growth via segmentation.